In [345]:
print ("Python Identifiers:")
print ("Class >>> Xxxx, others >>> xxxx")
print ("_xxxx indicates that the identifier is private.")
print ("__xxx indicates that the identifier is strongly private.")
print ("__xx__ indicates: A languagedefined special name")
import numpy as np
import pandas as pd
import os

Python Identifiers:
Class >>> Xxxx, others >>> xxxx
_xxxx indicates that the identifier is private.
__xxx indicates that the identifier is strongly private.
__xx__ indicates: A languagedefined special name


In [2]:
pd.__version__

u'0.23.1'

In [342]:
def DIR_CHECK_CREATE(Input_Path):
    if (not os.path.isdir(Input_Path)):
        print ("New Dir Made is" + Input_Path)
        os.mkdir(Input_Path)
    else: 
        print ('Dir Exists')
def Add_common_header(df,common_header):
    name_list=[common_header]*len(df.columns)
    tuples = list(zip(name_list,df.columns))
    df.columns = pd.MultiIndex.from_tuples(tuples)
    return df
FC_UP=2.0
q_value_less=0.05
FPKM_threshold=1.0
####################################################################################

def generate_Upregulated_Genes(df):
#### Filter
    df=df[(df['status']=='OK') & (df['q_value']<=q_value_less) & (df['value_2']>=FPKM_threshold) & ( (df['log2(fold_change)'] >= np.log2(FC_UP)))]
#### Output
    return df
####################################################################################
def generate_Downregulated_Genes(df):
#### Filter
    df=df[(df['status']=='OK') & (df['q_value']<=q_value_less) & (df['value_1']>=FPKM_threshold) & (df['log2(fold_change)']<= -np.log2(FC_UP))]   
#### Output
    return df
####################################################################################

In [378]:
def associate_motif_expression_excel(motif_gene_association_folder, expression_input):
    import glob
    import datetime
    from scipy import stats

    os.chdir(motif_gene_association_folder)
    
    INPUT_LIST = glob.glob('*.bed.gene_association.txt')
    writer = pd.ExcelWriter(motif_gene_association_folder+'/Motif_Associated_Genes_Summary_'+str(datetime.date.today())+ '.xlsx', engine='xlsxwriter')
    df_gene_exp = pd.read_csv(expression_input, sep='\t').rename(columns={'test_id':'Gene Name'}).drop(['gene_id','gene','test_stat'],axis=1)

    
    summary_df = pd.DataFrame(columns=['Motif_Name','wilcoxon_rank_test_pvalue','(Up-Down)/All']) ### wilcoxon signed rank test 
    i=0
    for name in INPUT_LIST[:]:
        df_motif_gene = pd.read_csv(name, sep='\t')
        df_merge = df_motif_gene.merge(df_gene_exp,on='Gene Name', how='inner')
        ## Gene Length Calculation
        gene_length = df_merge['locus'].str.split(pat=':',expand=True)[1].str.split(pat='-',expand=True).astype(int).apply(lambda x: x[1] - x[0], axis=1)
        df_merge.insert(5,'Distance to TES', df_merge['Distance to TSS'] - gene_length)
        ## Save a summary of motif associated with Expression
        Cond1_Name = df_merge['sample_1'][0]
        Cond2_Name = df_merge['sample_2'][0]
        df_merge.rename(columns={'value_1':Cond1_Name, 'value_2':Cond2_Name}).drop(['sample_1','sample_2'],
                axis=1).to_excel(writer, sheet_name=name[0:7], index=None)

        ### https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.ks_2samp.html
        ### KS 2 sample test
        ### https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html
        df_merge = df_merge[df_merge['status']=='OK']
        #print "pvalue:",stats.ks_2samp(df_merge['value_1'], df_merge['value_2'])[1] ## wilcoxon signed rank test 
        up_ratio = 1.0*(len(generate_Upregulated_Genes(df_merge)['Gene Name'].unique()) -  len(generate_Downregulated_Genes(df_merge)['Gene Name'].unique()))/len(df_merge['Gene Name'].unique())
        summary_df.loc[i,['Motif_Name','wilcoxon_rank_test_pvalue','(Up-Down)/All']] = [name[0:7], stats.wilcoxon(df_merge['value_1'], df_merge['value_2'])[1], up_ratio]
        i+=1
    up_ratio = 1.0*(len(generate_Upregulated_Genes(df_gene_exp)['Gene Name']) -  len(generate_Downregulated_Genes(df_gene_exp)['Gene Name']))/len(df_gene_exp['Gene Name'])
    summary_df.loc[i,['Motif_Name','wilcoxon_rank_test_pvalue','(Up-Down)/All']] = ['Overall', stats.wilcoxon(df_gene_exp['value_1'], df_gene_exp['value_2'])[1], up_ratio]
    summary_df.sort_values(by=['(Up-Down)/All'],ascending=False).to_excel(writer, sheet_name='Motif Ranking Test', index=None)
    writer.save()
    
    return 0




expression_input='/home/xli/cloud_research/PengGroup/XLi/Data/Haihui/CD8-HP/RNA_seq/CuffDiff_Jun2018/Cuffdiff_Results/WT_72h_vs_WT_0h/gene_exp.diff'
associate_motif_expression_excel(PATH_FOLDER, expression_input)

0

954

In [381]:
PATH_FOLDER='/home/data/www/Homer_Results/Motif_Results/2442_sum_vs_445_summ/knownResults'
#PATH_FOLDER='/home/data/www/Homer_Results/Motif_Results/1537_sum_vs_1387_sum/homerResults'
#expression_input='/home/xli/cloud_research/PengGroup/XLi/Data/Haihui/CD8-HP/RNA_seq/CuffDiff_Jun2018/Cuffdiff_Results/DKO_0h_vs_WT_0h/gene_exp.diff'
expression_input='/home/xli/cloud_research/PengGroup/XLi/Data/Haihui/CD8-HP/RNA_seq/CuffDiff_Jun2018/Cuffdiff_Results/WT_72h_vs_WT_0h/gene_exp.diff'
DIR_CHECK_CREATE(PATH_FOLDER)
#DIR_CHECK_CREATE(OUT_FOLDER)
INPUT_LIST=os.listdir(PATH_FOLDER)

import glob
import datetime
from scipy import stats
os.chdir(PATH_FOLDER)
INPUT_LIST = glob.glob('*.bed.gene_association.txt')
#writer = pd.ExcelWriter(PATH_FOLDER+'/Motif_Associated_Genes_Summary_'+str(datetime.date.today())+ '.xlsx', engine='xlsxwriter')


df_gene_exp = pd.read_csv(expression_input, sep='\t').rename(columns={'test_id':'Gene Name'}).drop(['gene_id','gene'],axis=1)



summary_df = pd.DataFrame(columns=['Motif_Name','pvalue', '(Up-Down)/All'])
i=0
for name in INPUT_LIST[:]:

    df_motif_gene = pd.read_csv(name, sep='\t')
    df_merge = df_motif_gene.merge(df_gene_exp,on='Gene Name', how='inner')
    ## Gene Length Calculation
    gene_length = df_merge['locus'].str.split(pat=':',expand=True)[1].str.split(pat='-',expand=True).astype(int).apply(lambda x: x[1] - x[0], axis=1)
    df_merge.insert(5,'Distance to TES', df_merge['Distance to TSS'] - gene_length)
    ## Save a summary of motif associated with Expression
    Cond1_Name = df_merge['sample_1'][0]
    Cond2_Name = df_merge['sample_2'][0]
    df_merge.rename(columns={'value_1':Cond1_Name, 'value_2':Cond2_Name}).drop(['sample_1','sample_2'],
            axis=1).to_excel(writer, sheet_name=name[0:7], index=None)
    
    ### https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.ks_2samp.html
    ### KS 2 sample test
    df_merge = df_merge[df_merge['significant']=='yes' ] #  [df_merge['status']=='OK']# 
    
    
    print 1.0*(len(generate_Upregulated_Genes(df_merge)) - len(generate_Downregulated_Genes(df_merge)))/len(df_merge[(df_merge['status']=='OK')])
    print name[0:7]," chi pvalue:",stats.chi2_contingency(df_merge[['value_1','value_2']])[1]
    print name[0:7]," pvalue:",stats.wilcoxon(df_merge['value_1'], df_merge['value_2'])[1]
    
    #print "MWN pvalue:",stats.mannwhitneyu(df_merge['value_1'], df_merge['value_2'], alternative='two-sided' )[1]
    
    up_ratio = 1.0*(len(generate_Upregulated_Genes(df_merge)['Gene Name'].unique()) -  len(generate_Downregulated_Genes(df_merge)['Gene Name'].unique()))/len(df_merge['Gene Name'].unique())
    summary_df.loc[i,['Motif_Name','pvalue','(Up-Down)/All']] = [name[0:7], stats.wilcoxon(df_merge['value_1'], df_merge['value_2'])[1], up_ratio]
    i+=1
    #print df_merge[['value_1','value_2']].describe()
    #print ""
    if i == 11:
        break
#df_tem = pd.DataFrame(df_merge['value_1'] / df_merge['value_2'], columns=['diff']).reset_index().drop('index',axis=1)
#df_tem.plot()
#summary_df.sort_values(by=['pvalue']).to_excel(writer, sheet_name='Kolmogorov-Smirnov Test', index=None)
#writer.save()
   

#https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test
#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.ks_2samp.html

Dir Exists
0.1375
known2.  chi pvalue: 2.930834482337056e-156
known2.  pvalue: 0.564897903925808
0.0540540540541
known9.  chi pvalue: 1.1487611325690116e-09
known9.  pvalue: 0.28745586378125043
0.137931034483
known6.  chi pvalue: 5.374338720104001e-23
known6.  pvalue: 0.39297131995431445
-0.04
known8.  chi pvalue: 7.416609149825929e-17
known8.  pvalue: 0.8611435378059807
0.145454545455
known10  chi pvalue: 2.52540370573794e-167
known10  pvalue: 0.5050505524571149
0.0602409638554
known5.  chi pvalue: 3.95241884899268e-56
known5.  pvalue: 0.7130360345331365
0.120481927711
known3.  chi pvalue: 3.6444452667961356e-146
known3.  pvalue: 0.570347235846639
0.0909090909091
known1.  chi pvalue: 3.517348586437878e-39
known1.  pvalue: 0.6990843376501483
0.107142857143
known7.  chi pvalue: 3.754889188224879e-201
known7.  pvalue: 0.572933079363372
0.126984126984
known4.  chi pvalue: 8.109320070726928e-141
known4.  pvalue: 0.8106140033576562


In [328]:
test1=[3,0,3,1,1,1]
test2=[0,3,0,0,1,10.1]
print "MWN pvalue:",stats.mannwhitneyu(test1, test2, alternative='two-sided' )[1]
#print stats.chi2_contingency( [test1, test2])[1]
print stats.chi2_contingency(df_merge[['value_1','value_2']])[1]
#df_merge[['value_1','value_2']].reset_index().drop('index',axis=1).iloc[0:39]

MWN pvalue: 0.6757614370951559
2.5095114321667592e-11


In [394]:
#summary_df
#len(generate_Upregulated_Genes(df_gene_exp)['Gene Name'])
#len(generate_Downregulated_Genes(df_gene_exp)['Gene Name'])
up_ratio = 1.0*(len(generate_Upregulated_Genes(df_gene_exp)['Gene Name']) -  len(generate_Downregulated_Genes(df_gene_exp)['Gene Name']))/len(df_gene_exp['Gene Name'])
up_ratio
#summary_df.loc[i,['Motif_Name','wilcoxon_rank_test_pvalue','(Up-Down)/All']] = ['Overall', stats.wilcoxon(df_gene_exp['value_1'], df_gene_exp['value_2'])[1], up_ratio]

0.019247630579098993

In [103]:

#from scipy import stats
#print "pvalue:",stats.mannwhitneyu(df_merge['value_1'], df_merge['value_2'], alternative='two-sided' )[1]
df_merge
#df_merge.rename(columns={'value_1':df_merge['sample_1'][0], 'value_2':df_merge['sample_2'][0]}).drop(['sample_1','sample_2'],axis=1)

#df_merge[['sample_1','sample_2','value_1','value_2']]

#df_merge[['value_1','value_2']]
#MWU, pvalue = ( stats.mannwhitneyu(all_data[0], all_data[1], alternative=None )[0], 
#                stats.mannwhitneyu(all_data[0], all_data[1], alternative='two-sided')[1])

KeyError: 0

In [192]:
motif_gene_association_input = PATH_FOLDER+'/motif1.bed.gene_association.txt'
df1 = pd.read_csv(PATH_FOLDER+'/motif1.bed.gene_association.txt', sep='\t')
df2 = pd.read_csv(PATH_FOLDER+'/gene_exp.diff', sep='\t')
df2 = df2.rename(columns={'test_id':'Gene Name'}).drop(['gene_id','gene'],axis=1)
df_merge = df1.merge(df2,on='Gene Name', how='inner')
gene_length = df_merge['locus'].str.split(pat=':',expand=True)[1].str.split(pat='-',expand=True).astype(int).apply(lambda x: x[1] - x[0], axis=1)

df_merge.insert(5,'Distance to TES', df_merge['Distance to TSS'] - gene_length)
df_merge.to_csv(motif_gene_association_input[:-4]+'_expression.txt',sep='\t', index=False)

In [188]:
gene_length = df_merge['locus'].str.split(pat=':',expand=True)[1].str.split(pat='-',expand=True).astype(int).apply(lambda x: x[1] - x[0], axis=1)

df_merge.insert(5,'Distance to TES', df_merge['Distance to TSS'] - gene_length)

In [189]:
df_merge.iloc[0:5,:]

,Chr,Start,End,Strand,Distance to TSS,Distance to TES,Gene Name,locus,sample_1,sample_2,status,value_1,value_2,log2(fold_change),test_stat,p_value,q_value,significant
0,chr12,86796454,86796463,-,-18382.0,-21751.0,Fos,chr12:86814850-86818219,WT_0h,WT_72h,OK,649.62900,0.625755,-10.019800,-21.44400,0.00005,0.000205,yes
1,chr2,32307037,32307046,+,-51.0,-41041.0,Slc25a25,chr2:32270006-32310996,WT_0h,WT_72h,OK,12.37130,8.281480,-0.579039,-2.16210,0.00025,0.000907,yes
2,chr8,85966561,85966570,+,-4.0,-23909.0,Scoc,chr8:85958390-85982295,WT_0h,WT_72h,OK,9.62266,7.466260,-0.366050,-1.32254,0.02145,0.045512,yes
3,chr8,85966600,85966609,+,-43.0,-23948.0,Scoc,chr8:85958390-85982295,WT_0h,WT_72h,OK,9.62266,7.466260,-0.366050,-1.32254,0.02145,0.045512,yes
4,chr2,109857942,109857951,-,-28.0,-32691.0,Ccdc34,chr2:109857973-109890636,WT_0h,WT_72h,OK,3.00898,8.434390,1.487010,4.71044,0.00005,0.000205,yes
